In [ ]:
pip install plotly

# Importing Environment Variables

In [1]:
import os
import openai
import sys

sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
google_api_key = os.environ['GOOGLE_API_KEY']
google_cse_id=os.environ['GOOGLE_CSE_ID']

# Spark Session Initialization

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("English_SDK_Demo_with_FIFA_WC_Final_Results") \
    .getOrCreate()

# OpenAI Initialization

In [3]:
from pyspark_ai import SparkAI
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
spark_ai = SparkAI(llm, spark_session=spark, verbose=True)
# The following activates partial functions for Spark DataFrame
# so, the DataFrame itself can be used to call the new Gen AI functions.
spark_ai.activate()  

INFO: The pyspark.sql.connect.dataframe module could not be imported. This might be due to your PySpark version being below 3.4.


# Data Ingestion

In [4]:
wc_df = spark_ai.create_df("https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals")
wc_df.show(n=5)

INFO: Parsing URL: https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW world_cup_finals AS 
SELECT 
    year,
    winners,
    runners_up,
    venue,
    location,
    attendance
FROM 
    (VALUES 
        (1930, 'Uruguay', 'Argentina', 'Estadio Centenario', 'Montevideo, Uruguay', 68346),
        (1934, 'Italy', 'Czechoslovakia', 'Stadio Nazionale PNF', 'Rome, Italy', 55000),
        (1938, 'Italy', 'Hungary', 'Stade Olympique de Colombes', 'Paris, France', 45000),
        (1950, 'Uruguay', 'Brazil', 'Maracanã Stadium', 'Rio de Janeiro, Brazil', 173850),
        (1954, 'West Germany', 'Hungary', 'Wankdorf Stadium', 'Bern, Switzerland', 62500),
        (1958, 'Brazil', 'Sweden', 'Råsunda Stadium', 'Solna, Sweden', 49737),
        (1962, 'Brazil', 'Czechoslovakia', 'Estadio Nacional', 'Santiago, Chile', 68679),
        (1966, 'England', 'West Germany', 'Wembley Stadium', 'London, England', 96924),
        (1970, 'Bra

# Data Transformarions

In [5]:
att_df = wc_df.ai.transform("select only year and attendance order by year ascending")
att_df.show(n=5)

INFO: SQL query for the transform:
SELECT year, attendance
FROM temp_view_for_transform
ORDER BY year ASC



+----+----------+
|year|attendance|
+----+----------+
|1930|     68346|
|1934|     55000|
|1938|     45000|
|1950|    173850|
|1954|     62500|
+----+----------+
only showing top 5 rows



In [6]:
att_df2 = att_df.ai.transform("add a new column called 'serial_no' with year and attendance order by year ascending")
att_df3 = att_df2.ai.transform ("get attendance from previous serial_no and add that as a new column called 'prev_attd' along with year and attendance order by year ascending")
att_df3.show(n=5)

INFO: SQL query for the transform:
SELECT year, attendance, ROW_NUMBER() OVER (ORDER BY year ASC) as serial_no
FROM temp_view_for_transform
ORDER BY year ASC

INFO: SQL query for the transform:
SELECT 
  year, 
  attendance, 
  LAG(attendance) OVER (ORDER BY year) AS prev_attd
FROM 
  temp_view_for_transform
ORDER BY 
  year ASC

+----+----------+---------+
|year|attendance|prev_attd|
+----+----------+---------+
|1930|     68346|     null|
|1934|     55000|    68346|
|1938|     45000|    55000|
|1950|    173850|    45000|
|1954|     62500|   173850|
+----+----------+---------+
only showing top 5 rows



In [11]:
@spark_ai.udf
def attendance_diff_w_prev_year(attendance: int, prev_attd: int) -> int:
    """Calculate 'attendance' difference with 'prev_attd' as percentage change. Return as integer"""
    ...
    
spark.udf.register("attendance_diff_w_prev_year", attendance_diff_w_prev_year)
att_df3.createOrReplaceTempView("att_df3")

df = spark.sql("select year, attendance, prev_attd, attendance_diff_w_prev_year(attendance, prev_attd) as attendance_diff_percentage from att_df3")

INFO: Creating following Python UDF:
def attendance_diff_w_prev_year(attendance, prev_attd) -> int:
    if attendance is not None and prev_attd is not None:
        return int(((attendance - prev_attd) / prev_attd) * 100)



# Plot

In [7]:
att_df.ai.plot("attendance over year, print all years")

INFO: To visualize the result of the `df` dataframe using Plotly, we can follow these steps:

1. Import the necessary libraries: `pyspark.sql` for Spark SQL operations and `plotly.graph_objects` for creating the plot.
2. Convert the Spark DataFrame `df` into a Pandas DataFrame using the `toPandas()` method.
3. Create a line plot using Plotly, with the `year` column as the x-axis and the `attendance` column as the y-axis.
4. Display the plot directly using the `show()` method.

Here's the code that accomplishes the above steps:


```
import pyspark.sql.functions as F
import plotly.graph_objects as go

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Create line plot using Plotly
fig = go.Figure(data=go.Scatter(x=pandas_df['year'], y=pandas_df['attendance'], mode='lines'))

# Set plot title and axis labels
fig.update_layout(title='Attendance over Year', xaxis_title='Year', yaxis_title='Attendance')

# Display the plot
fig.show()
```

This code will create a line

In [8]:
winner_df = wc_df.ai.transform ("select winner, rename 'West Germany' as 'Germany'")
winner_df.ai.plot("pie chart with top 6 winners")

INFO: SQL query for the transform:
SELECT 
  CASE 
    WHEN winners = 'West Germany' THEN 'Germany'
    ELSE winners
  END AS winner
FROM 
  temp_view_for_transform

INFO: To visualize the result of the `df` dataframe using Plotly, we can create a pie chart with the top 6 winners. Here's the Python code to achieve this:


```
import plotly.express as px

# Assuming `df` is the Spark DataFrame with the output columns: winner
# Convert the Spark DataFrame to a Pandas DataFrame
df_pandas = df.toPandas()

# Count the occurrences of each winner
winner_counts = df_pandas['winner'].value_counts()

# Get the top 6 winners
top_6_winners = winner_counts.head(6)

# Create a pie chart using Plotly
fig = px.pie(top_6_winners, values=top_6_winners.values, names=top_6_winners.index)

# Display the plot
fig.show()
```

This code first converts the Spark DataFrame `df` to a Pandas DataFrame `df_pandas`. Then, it counts the occurrences of each winner using the `value_counts()` method. Next, it selects t

In [9]:
runner_df = wc_df.ai.transform ("select runner, rename 'West Germany' as 'Germany'")
#runners_up
runner_df.ai.plot("pie chart with top 10 runner ups")

INFO: SQL query for the transform:
SELECT 
  CASE 
    WHEN runners_up = 'West Germany' THEN 'Germany'
    ELSE runners_up
  END AS runner
FROM 
  temp_view_for_transform

INFO: To visualize the result of the `df` dataframe using Plotly, we can create a pie chart with the top 10 runner-ups. Here's the Python code to achieve this:


```
import plotly.express as px

# Create a new dataframe with the count of each runner-up
runner_counts = df.groupBy('runner').count().orderBy('count', ascending=False).limit(10)

# Convert the dataframe to a Pandas dataframe
runner_counts_pd = runner_counts.toPandas()

# Create the pie chart using Plotly
fig = px.pie(runner_counts_pd, values='count', names='runner', title='Top 10 Runner-ups')

# Display the plot
fig.show()
```

In this code, we first group the `df` dataframe by the `runner` column and count the occurrences of each runner-up. We then order the result in descending order and limit it to the top 10 runner-ups.

Next, we convert the resulting 

In [32]:
finalist_df.show(5)

INFO: SQL query for the transform:
SELECT year, winners AS finalists
FROM temp_view_for_transform
UNION
SELECT year, runners_up AS finalists
FROM temp_view_for_transform
ORDER BY year ASC



+----+--------------+
|year|     finalists|
+----+--------------+
|1930|       Uruguay|
|1930|     Argentina|
|1934|         Italy|
|1934|Czechoslovakia|
|1938|       Hungary|
+----+--------------+
only showing top 5 rows



In [38]:
wc_clean_df = wc_df.ai.transform ("rename 'West Germany' as 'Germany' in winners and runners_up")
finalist_df = wc_clean_df.ai.transform ("""select year, winner union 
                                  select year, runner column. 
                                  Name the new column as finalists,
                                  Order year ascending""")
finalist_df2 = finalist_df.ai.transform ("select finalists")
finalist_df2.ai.plot("bar chart of finalists from most to least")

INFO: SQL query for the transform:
SELECT 
    year,
    CASE WHEN winners = 'West Germany' THEN 'Germany' ELSE winners END AS winners,
    CASE WHEN runners_up = 'West Germany' THEN 'Germany' ELSE runners_up END AS runners_up,
    venue,
    location,
    attendance
FROM 
    temp_view_for_transform

INFO: SQL query for the transform:
SELECT year, winners AS finalists
FROM temp_view_for_transform
UNION
SELECT year, runners_up AS finalists
FROM temp_view_for_transform
ORDER BY year ASC

INFO: SQL query for the transform:
SELECT finalists FROM temp_view_for_transform



INFO: To visualize the result of `df` using plotly, we can follow these steps:

1. Import the necessary libraries:

```
import plotly.express as px
import pyspark.sql.functions as F
```

2. Group the dataframe by the `finalists` column and count the number of occurrences:

```
df_grouped = df.groupBy("finalists").agg(F.count("*").alias("count"))
```

3. Sort the dataframe by the count in descending order:

```
df_sorted = df_grouped.orderBy(F.desc("count"))
```

4. Create the bar chart using plotly:

```
fig = px.bar(df_sorted.toPandas(), x="finalists", y="count", color="finalists")
```

5. Display the plot:

```
fig.show()
```

Here's the complete code:


```
import plotly.express as px
import pyspark.sql.functions as F

df_grouped = df.groupBy("finalists").agg(F.count("*").alias("count"))
df_sorted = df_grouped.orderBy(F.desc("count"))

fig = px.bar(df_sorted.toPandas(), x="finalists", y="count", color="finalists")
fig.show()
```

This code will create a bar chart showing the number 

# Explain

## Explaing a Spark DataFrame

In [39]:
finalist_df2.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [finalists#579]
   +- Sort [year#12 ASC NULLS FIRST], true, 0
      +- Exchange rangepartitioning(year#12 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=1857]
         +- HashAggregate(keys=[year#12, finalists#579], functions=[])
            +- Exchange hashpartitioning(year#12, finalists#579, 200), ENSURE_REQUIREMENTS, [plan_id=1854]
               +- HashAggregate(keys=[year#12, finalists#579], functions=[])
                  +- Union
                     :- LocalTableScan [year#12, finalists#579]
                     +- LocalTableScan [year#587, finalists#588]




## Explaining using Open AI

In [40]:
finalist_df2.ai.explain()

"In summary, this dataframe is retrieving the finalists of the World Cup for each year. It combines the winners and runners-up of each year, replacing 'West Germany' with 'Germany' in the process. The results are sorted by year."

# Verification

In [44]:
finalist_df2.ai.verify("expect Germany is total 8 finalists")

INFO: Generated code:
def has_8_finalists(df) -> bool:
    # Filter the DataFrame to only include rows where finalists is equal to Germany
    germany_finalists = df.filter(df.finalists == "Germany")

    # Get the number of rows where finalists is equal to Germany
    num_germany_finalists = germany_finalists.count()

    # Check if the number of Germany finalists is equal to 8
    if num_germany_finalists == 8:
        return True
    else:
        return False

result = has_8_finalists(df)

INFO: 
Result: True


# In-memory Cache

In [45]:
spark_ai.commit()